In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
#Advanced Fork-Join Example 12#Complex fork-join with 2 parallel branches
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
queue3 = Queue(model,'Queue3',SchedStrategy.FCFS)
queue4 = Queue(model,'Queue4',SchedStrategy.FCFS)
queue5 = Queue(model,'Queue5',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(2.0))
queue3.set_service(jobclass1, Exp(1.0))
queue4.set_service(jobclass1, Exp(3.0))
queue5.set_service(jobclass1, Exp(0.8))
delay.set_service(jobclass1, Exp(0.5))

In [5]:
# Set routing matrix with complex serial routing
P = model.init_routing_matrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
# Complex serial routing:
P.set(jobclass1, jobclass1, queue1, queue4, 1.0)  # queue1 -> queue4
P.set(jobclass1, jobclass1, queue4, queue5, 1.0)  # queue4 -> queue5  
P.set(jobclass1, jobclass1, queue5, join, 1.0)    # queue5 -> join
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  # queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)    # queue3 -> join
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/1881338300358790791/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 0.997908s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.5268  1.5268  2.0031  2.0031  0.7608  0.7605
1  Queue1   class1  2.4714  0.7577  3.2755  1.6378  0.7605  0.7662
2  Queue2   class1  0.5817  0.3711  0.7558  0.3779  0.7605  0.7599
3  Queue3   class1  2.8414  0.7579  3.5921  1.7960  0.7599  0.7655
4  Queue4   class1  0.3381  0.2501  0.4457  0.2229  0.7662  0.7576
5  Queue5   class1  5.4035  0.9519  7.0996  3.5498  0.7576  0.7620
6    Join   class1  5.3121  0.0000  3.4381  3.4381  1.5305  0.7607
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.083628s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.3867  1.3867  2.0000  2.0000  0.6934  0.6934
1  Queue1   class1  2.1420  0.6933  3.0894  1.5447  0.6934  0.6933
2